In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/football-match-probability-prediction/sample_submission.csv
/kaggle/input/football-match-probability-prediction/train_target_and_scores.csv
/kaggle/input/football-match-probability-prediction/train.csv
/kaggle/input/football-match-probability-prediction/test.csv


In [2]:
#import modules
from sklearn import metrics, model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

#loading the data
train = pd.read_csv('/kaggle/input/football-match-probability-prediction/train.csv')
test = pd.read_csv('/kaggle/input/football-match-probability-prediction/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Features Construction:
* is_cup converts to 0/1
* matchday converts to time interval
* previous n_game form: goal difference, average points, average goal difference
* coach change

In [3]:
train.is_cup.value_counts()

False    101856
True       9081
Name: is_cup, dtype: int64

In [4]:
a=train.is_cup.apply(lambda x: np.multiply(x, 1))
a

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
110933    0.0
110934    0.0
110935    0.0
110936    0.0
110937    0.0
Name: is_cup, Length: 110938, dtype: float64

In [5]:
a.value_counts()

0.0    101856
1.0      9081
Name: is_cup, dtype: int64

In [6]:
def is_cup_conversion(df):
    '''convert is_cup column from boolean to 0/1'''
    df.is_cup = df.is_cup.apply(lambda x: np.multiply(x, 1) )
    return df

In [7]:
is_cup_conversion(train)

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,...,away_team_history_league_id_1,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8,away_team_history_league_id_9,away_team_history_league_id_10
0,11906497,away,Newell's Old Boys,River Plate,2019-12-01 00:45:00,Superliga,636,0.0,468196.0,468200.0,...,1122.0,642.0,636.0,636.0,636.0,1122.0,636.0,642.0,636.0,1122.0
1,11984383,home,Real Estelí,Deportivo Las Sabanas,2019-12-01 01:00:00,Primera Division,752,0.0,516788.0,22169161.0,...,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0
2,11983301,draw,UPNFM,Marathón,2019-12-01 01:00:00,Liga Nacional,734,0.0,2510608.0,456313.0,...,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0
3,11983471,away,León,Morelia,2019-12-01 01:00:00,Liga MX,743,0.0,1552508.0,465797.0,...,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0,746.0,743.0
4,11883005,home,Cobán Imperial,Iztapa,2019-12-01 01:00:00,Liga Nacional,705,0.0,429958.0,426870.0,...,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110933,18030016,draw,Zamora Fútbol Club,Hermanos Colmenares,2021-04-30 23:00:00,Primera Division,800,0.0,7708122.0,37564088.0,...,800.0,800.0,1101.0,1101.0,803.0,803.0,803.0,803.0,803.0,803.0
110934,18030096,away,Royal Pari,Oriente Petrolero,2021-04-30 23:00:00,Liga De Futbol Prof,1098,0.0,465833.0,16488543.0,...,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0
110935,17715497,draw,São Bernardo,Água Santa,2021-04-30 23:00:00,Paulista A2,1314,0.0,8416491.0,528562.0,...,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1746.0,1746.0
110936,17944153,away,Everton,La Serena,2021-05-01 00:00:00,Primera Division,663,0.0,6973085.0,471757.0,...,663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0


In [8]:
train['home_has_coach_change']=pd.Series(any([(train.loc[r, 'home_team_coach_id']!=train.loc[r, f'home_team_history_coach_{i}']) & 
                                              (np.isnan(train.loc[r, f'home_team_history_coach_{i}'])==False) for i in range(1,11) ]) 
                                         for r in train.index)

In [9]:
train.home_has_coach_change.value_counts()

False    79744
True     31194
Name: home_has_coach_change, dtype: int64

In [10]:
train[train.home_has_coach_change==True]

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,...,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8,away_team_history_league_id_9,away_team_history_league_id_10,home_has_coach_change
6,11974170,home,Team Wellington,Canterbury United,2019-12-01 01:00:00,Premiership,1055,0.0,37347427.0,37348899.0,...,1055.0,1055.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,True
31,11954687,away,Happy Valley,Southern District,2019-12-01 06:30:00,Premier League,1001,0.0,37288526.0,1552671.0,...,1001.0,1001.0,1001.0,1001.0,1001.0,1528.0,1001.0,1528.0,1001.0,True
34,11864730,home,Andijan,Qizilqum,2019-12-01 07:00:00,Professional Football League,1349,0.0,35683630.0,1553013.0,...,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,True
35,11864735,draw,Kokand-1912,Navbahor,2019-12-01 07:00:00,Professional Football League,1349,0.0,37252510.0,466543.0,...,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,1349.0,True
38,11775442,home,Beijing Guoan,Shandong Luneng,2019-12-01 07:00:00,Super League,989,0.0,523953.0,9302752.0,...,989.0,995.0,989.0,989.0,989.0,989.0,995.0,989.0,989.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110930,17715491,away,Juventus,Oeste,2021-04-30 23:00:00,Paulista A2,1314,0.0,529523.0,460181.0,...,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,651.0,651.0,True
110933,18030016,draw,Zamora Fútbol Club,Hermanos Colmenares,2021-04-30 23:00:00,Primera Division,800,0.0,7708122.0,37564088.0,...,800.0,1101.0,1101.0,803.0,803.0,803.0,803.0,803.0,803.0,True
110934,18030096,away,Royal Pari,Oriente Petrolero,2021-04-30 23:00:00,Liga De Futbol Prof,1098,0.0,465833.0,16488543.0,...,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,1098.0,True
110935,17715497,draw,São Bernardo,Água Santa,2021-04-30 23:00:00,Paulista A2,1314,0.0,8416491.0,528562.0,...,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1746.0,1746.0,True


In [11]:
def has_coach_change(df):
    '''add features indicating whether home and away teams have changed coach'''
    df['home_has_coach_change']=pd.Series(any([(df.loc[r, 'home_team_coach_id']!=df.loc[r, f'home_team_history_coach_{i}']) & 
                                              (np.isnan(df.loc[r, f'home_team_history_coach_{i}'])==False) for i in range(1,11) ]) 
                                         for r in df.index)
    df['away_has_coach_change']=pd.Series(any([(df.loc[r, 'away_team_coach_id']!=df.loc[r, f'away_team_history_coach_{i}']) & 
                                              (np.isnan(df.loc[r, f'away_team_history_coach_{i}'])==False) for i in range(1,11) ]) 
                                         for r in df.index)
    return df

In [12]:
def add_match_interval(df):
    '''add features indicating the time gap between the match day of each previous match for the home and away teams,
    as an reflection of fatique factor'''
    for i in range(1,11):
        df[f'home_team_history_match_interval_{i}'] = pd.Series(
            (pd.Timestamp(train.loc[r, 'match_date'])-pd.Timestamp(train.loc[r, f'home_team_history_match_date_{i}']))
            /np.timedelta64(1, "D") 
            for r in train.index)
        df[f'away_team_history_match_interval_{i}'] = pd.Series(
            (pd.Timestamp(train.loc[r, 'match_date'])-pd.Timestamp(train.loc[r, f'away_team_history_match_date_{i}']))
            /np.timedelta64(1, "D") 
            for r in train.index)
    return df

In [13]:
def result_point(goal_difference):
    if goal_difference > 0:
        return 3
    elif goal_difference==0:
        return 1
    else:
        return 0

In [14]:
def add_form_features(df):
    '''adding form features for home and away teams, including rating difference and goal difference for each previous match,
    and the average points got, average goal scored, average goal conceded and average goal difference up to that match as previous n-match form'''
    for i in range(1,11):
        df[f'home_team_history_goal_difference_{i}']=df[f'home_team_history_goal_{i}']-df[f'home_team_history_opponent_goal_{i}']
        df[f'away_team_history_goal_difference_{i}']=df[f'away_team_history_goal_{i}']-df[f'away_team_history_opponent_goal_{i}']
        df[f'home_team_history_rating_difference_{i}']=df[f'home_team_history_rating_{i}']-df[f'home_team_history_opponent_rating_{i}']
        df[f'away_team_history_rating_difference_{i}']=df[f'away_team_history_rating_{i}']-df[f'away_team_history_opponent_rating_{i}']
        df[f'home_team_history_average_goal_scored_{i}']=pd.Series(pd.Series(df.loc[r, f'home_team_history_goal_{j}'] for j in range (1,i+1)).mean() for r in df.index)
        df[f'away_team_history_average_goal_scored_{i}']=pd.Series(pd.Series(df.loc[r, f'away_team_history_goal_{j}'] for j in range (1,i+1)).mean() for r in df.index)
        df[f'home_team_history_average_goal_conceded_{i}']=pd.Series(pd.Series(df.loc[r, f'home_team_history_opponent_goal_{j}'] for j in range (1,i+1)).mean() for r in df.index)
        df[f'away_team_history_average_goal_conceded_{i}']=pd.Series(pd.Series(df.loc[r, f'away_team_history_opponent_goal_{j}'] for j in range (1,i+1)).mean() for r in df.index)
        df[f'home_team_history_average_goal_difference_{i}']=df[f'home_team_history_average_goal_scored_{i}']-df[f'home_team_history_average_goal_conceded_{i}']
        df[f'away_team_history_average_goal_difference_{i}']=df[f'away_team_history_average_goal_scored_{i}']-df[f'away_team_history_average_goal_conceded_{i}']
        df[f'home_team_history_average_point_{i}']=pd.Series(pd.Series(df.loc[r, f'home_team_history_goal_difference_{j}'] for j in range (1,i+1)).apply(result_point).mean() for r in df.index)
        df[f'away_team_history_average_point_{i}']=pd.Series(pd.Series(df.loc[r, f'away_team_history_goal_difference_{j}'] for j in range (1,i+1)).apply(result_point).mean() for r in df.index)
        return df